In [1]:
!pip install catboost
!unzip file.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.8 MB/s eta 0:00:00
Archive:  file.zip
   creating: baseline/
  inflating: baseline/baseline_forecast.csv  
  inflating: baseline/baseline_solution.py  
 extracting: baseline/__init__.py    
   creating: data/
  inflating: data/train.csv          
   creating: docs/
   creating: docs/images/
  inflating: docs/images/decline_curve.png  
  inflating: docs/images/spatial_field.png  
  inflating: docs/Описание задачи Сириус.docx  
  inflating: README.md               
  inflating: requirements.txt        
  inflating: utils.py                


In [1]:
import numpy as np
from catboost import Pool, CatBoostRegressor

from tqdm import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

In [2]:
df = pd.read_csv('data/train.csv')
df = df.drop_duplicates()

In [3]:
def interpolate(df, borehole):
    value_df = df[df['Номер скважины'] == borehole].copy()
    df[df['Номер скважины'] == borehole] = value_df.interpolate().bfill().ffill()

for borel in df['Номер скважины'].unique():
    interpolate(df, borel)

In [4]:
df = df.fillna(df.mean())

<ipython-input-4-04e87d67d337>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [11]:
lst_of_join = []

def add_ewm(df, borehole):
    value_df = df[df['Номер скважины'] == borehole].copy()
    means = [value_df.ewm(com=tmp).mean() for tmp in range(1, 100, 15)]
    for shift in tqdm(range(1, 100, 20)):
        for cnt, mean in enumerate(means):
            value_df = value_df.join(mean.shift(shift), rsuffix=f"{shift} {cnt}mean") 
    return value_df.ffill().bfill()

ewm_df = pd.concat([add_ewm(df, borel) for borel in df['Номер скважины'].unique()])

100%|██████████| 5/5 [00:00<00:00, 26.32it/s]


In [6]:
ewm_df.datetime

0        1990-08-01
1        1990-08-02
2        1990-08-03
3        1990-08-04
4        1990-08-05
            ...    
67131    1992-04-07
67132    1992-04-08
67133    1992-04-09
67134    1992-04-10
67135    1992-04-11
Name: datetime, Length: 66545, dtype: object

In [12]:
import datetime
ewm_df.datetime = ewm_df.datetime.apply(lambda dt: datetime.datetime.strptime(dt, "%Y-%m-%d").date())

In [13]:
ewm_df[ewm_df.datetime < datetime.datetime.strptime("1991-11-11", "%Y-%m-%d").date()]

,datetime,Номер скважины,Дебит нефти,Давление забойное,x,y,Объем жидкости,Объем нефти,Активная мощность (ТМ),Время работы (ТМ),...,Газовый фактор рабочий (ТМ)81 6mean,Давление буферное81 6mean,Давление забойное от Hд81 6mean,Давление забойное от Pпр81 6mean,Давление линейное (ТМ)81 6mean,Давление на входе ЭЦН (ТМ)81 6mean,Дебит газа (ТМ)81 6mean,Дебит газа попутного81 6mean,Дебит жидкости (ТМ)81 6mean,Коэффициент мощности (ТМ)81 6mean
0,1990-08-01,0,19.939,39.064454,18670.86,5714.86,788.311000,757.568000,50.851351,9.600833,...,239.555000,20.000000,33.913336,39.064454,1.887872,30.293676,3843.746667,24.250000,24.250000,98.534314
1,1990-08-02,0,19.939,39.064454,18670.86,5714.86,788.311000,757.568000,50.851351,9.600833,...,239.555000,20.000000,33.913336,39.064454,1.887872,30.293676,3843.746667,24.250000,24.250000,98.534314
2,1990-08-03,0,21.172,39.064487,18670.86,5714.86,788.311000,757.568000,52.353846,9.600417,...,239.555000,20.000000,33.913336,39.064454,1.887872,30.293676,3843.746667,24.250000,24.250000,98.534314
3,1990-08-04,0,22.529,38.965297,18670.86,5714.86,788.311000,757.568000,51.242424,9.600000,...,239.555000,20.000000,33.913336,39.064454,1.887872,30.293676,3843.746667,24.250000,24.250000,98.534314
4,1990-08-05,0,22.529,38.766822,18670.86,5714.86,788.311000,757.568000,50.910256,9.563889,...,239.555000,20.000000,33.913336,39.064454,1.887872,30.293676,3843.746667,24.250000,24.250000,98.534314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66978,1991-11-06,105,6.856,120.324556,14230.22,22456.04,15533.807355,270.271710,244.182540,24.000000,...,63.968032,11.984973,133.868314,113.562187,9.973715,31.480978,4257.673566,502.732222,495.728830,70.299711
66979,1991-11-07,105,6.861,120.323609,14230.22,22456.04,15538.804903,273.017806,244.070175,24.000000,...,63.968385,11.985139,133.719925,113.494128,9.986417,31.445066,4257.673566,502.646878,495.720786,70.324832
66980,1991-11-08,105,6.897,120.322405,14230.22,22456.04,15543.802452,275.763903,244.283824,24.000000,...,63.968734,11.985303,133.573171,113.426048,9.995781,31.408051,4257.673566,502.560283,495.710624,70.352147
66981,1991-11-09,105,6.892,120.315743,14230.22,22456.04,15548.800000,278.510000,244.354286,24.000000,...,63.969079,11.985465,133.428066,113.358729,10.008467,31.372549,4257.673566,502.487898,495.713817,70.376634


In [38]:
import tempfile

def find_the_best_model(X_train, y_train, X_test, y_test, X_final, y_final):
    train_pool = Pool(X_train,
                  y_train)

    test_pool = Pool(X_test,
                    y_test)
    final_pool = Pool(X_final,
                      y_final)

    the_best_rmse = 1e6
    the_best_iterations = 0
    the_best_depth = 0
    for count_of_iterations in tqdm(range(5, 11)):
        for level_of_depth in range(4,9):
            model=CatBoostRegressor(iterations=count_of_iterations,
                                depth=level_of_depth,
                                verbose=False,
                                learning_rate=0.1,)
        model.fit(train_pool)
        rmse = np.mean((y_test - model.predict(test_pool))**2)**0.5
        if rmse < the_best_rmse:
            the_best_rmse = rmse
            the_best_iterations = count_of_iterations
            the_best_depth = level_of_depth
            
    model = CatBoostRegressor(iterations=the_best_iterations,
                          depth=the_best_depth, 
                          verbose=False,
                          learning_rate=0.1,)
    return model.fit(final_pool)

def create_y(df, borehole, horizon):
    value_df = df[df['Номер скважины'] == borehole].copy()
    value_df['target'] = value_df.shift(-horizon)['Дебит нефти']
    df[df['Номер скважины'] == borehole] = value_df

def get_predictor(horizon):
    print("start", horizon)
    ewm_copy = ewm_df.copy()
    ewm_copy['target'] = 0
    for borel in tqdm(df['Номер скважины'].unique()):
        value_df = ewm_copy[ewm_copy['Номер скважины'] == borel].copy()
        value_df['target'] = value_df.shift(-horizon)['Дебит нефти']
        ewm_copy[ewm_copy['Номер скважины'] == borel] = value_df

    ewm_copy = ewm_copy.dropna()

    train_df = ewm_copy[ewm_copy.datetime < datetime.datetime.strptime("1991-11-11", "%Y-%m-%d").date()].copy()
    test_df = ewm_copy[ewm_copy.datetime >= datetime.datetime.strptime("1991-11-11", "%Y-%m-%d").date()].copy()
    X_train = train_df.drop(['target', 'datetime'], axis=1)
    X_test = test_df.drop(['target', 'datetime'], axis=1)
    X_final = ewm_copy.drop(['target', 'datetime'], axis=1)
    y_train = train_df['target']
    y_test = test_df['target']
    y_final = ewm_copy['target']
    return find_the_best_model(X_train, y_train, X_test, y_test, X_final, y_final)

In [34]:
ewm_df['Номер скважины'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105])

In [29]:
df['Номер скважины'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105])

In [31]:
def create_models():
    predictors = []
    # 90 / 30 = 3
    for days in range(1, 4):
        predictors.append(get_predictor(days*30))
    return predictors

In [39]:
predictors = create_models()

start 30


100%|██████████| 6/6 [00:46<00:00,  7.77s/it]


start 60


100%|██████████| 6/6 [00:46<00:00,  7.71s/it]


start 90


100%|██████████| 6/6 [00:46<00:00,  7.78s/it]


In [70]:
datetime_index = [(datetime.datetime(1992, 4, 11) + datetime.timedelta(days=dt)).date() for dt in range(90)]

def predict_future(borehole):
    res = []
    days = []
    tmp_df = ewm_df[ewm_df['Номер скважины'] == borehole].copy()
    for iter in range(0, 3):
        for horizon in (range(0, 30)):
            day = datetime.datetime(1992, 4, 11) - datetime.timedelta(days=30*(iter+1)-horizon)
            res.append(predictors[iter].predict(tmp_df[tmp_df.datetime == day.date()].copy().drop(['datetime'], axis=1))[0])
            days.append(datetime.datetime(1992, 4, 11) + datetime.timedelta(days=iter*30+horizon))
    ans_df = pd.DataFrame({'datetime': days, 'forecast': res, 'Номер скважины': borehole})
    ans_df['forecast'] = ans_df['forecast'].ewm(span=200).mean()
    return ans_df


In [72]:
preds = []
for borel in tqdm(df['Номер скважины'].unique()):
    preds.append(predict_future(borel))

100%|██████████| 106/106 [08:55<00:00,  5.05s/it]


In [ ]:
output_df=pd.concat(preds)

In [ ]:
output_df.to_csv('Volkova_forecast.csv',index=False, encoding="utf-8")